# **Week - 3 : Classification**

## **Import Dependencies**

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import kaggle
import requests

import os
import pathlib
import warnings
warnings.filterwarnings('ignore')

### **Download the Dataset**

In [ ]:
# Check if the dataset was already downloaded

downloadedFilePath = pathlib.Path.cwd() / "WA_Fn-UseC_-Telco-Customer-Churn.csv"
metadataDir = pathlib.Path.cwd() / "metadata"

if not (downloadedFilePath.is_file() and metadataDir.is_dir()):
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('blastchar/telco-customer-churn', unzip=True)
    kaggle.api.dataset_metadata('blastchar/telco-customer-churn', path='metadata')
else:
    print(f"Both the metadata directory and the dataset for this module already exist.")

: 

In [ ]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

: 

In [ ]:
df.head().T

: 

## **Data Preparation**

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.select_dtypes('object').columns)
for column in categorical_columns:
    df[column] = df[column].str.lower().str.replace(' ', '_')
df.info()

: 

In [ ]:
tc = pd.to_numeric(df.totalcharges, errors='coerce')
# df[tc.isnull()][['customerid', 'totalcharges']]

df.totalcharges = df.totalcharges.fillna(0)

: 

In [ ]:
df.churn = (df.churn == 'yes').astype(int)

: 

In [ ]:
df.churn

: 

## **Setting up the validation framework**

In [ ]:
from sklearn.model_selection import train_test_split

df_fulltrain, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [ ]:
len(df_fulltrain), len(df_test)

## **Eploratory Data Analysis**

## **Feature Importance: Churn Rate and Risk Ratio**

## **Feature Importance: Mutual Information**

## **Feature Importance: Correlation**

## **One-Hot Encoding**

## **Logistic Regression**

## **Training Logistic Regression with Scikit-Learn**